In [ ]:
%pip install -q nbformat plotly

import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML

file_path = "./model_risk_signals.csv"
df = pd.read_csv(file_path)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.rename(columns={'Model_signals': 'model_signals'})

df['EMA12'] = df['price'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['price'].ewm(span=26, adjust=False).mean()

df['MACD'] = df['EMA12'] - df['EMA26']

df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

df['Histogram'] = df['MACD'] - df['Signal']

df[['price', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram']].head()

initial_capital = 1000
capital = initial_capital
cash = initial_capital
in_trade = False
entry_price = 0
balance_history = [initial_capital]
trades = []

for i in range(1, len(df)):
    # Enter a short trade
    if df['Histogram'].iloc[i-1] >= 0 and df['Histogram'].iloc[i] < 0 and not in_trade:
        in_trade = True
        entry_price = df['price'].iloc[i]
        amount_to_trade = 0.5 * cash
        cash -= amount_to_trade
    
    # Exit the short trade
    elif df['Histogram'].iloc[i-1] < 0 and df['Histogram'].iloc[i] >= 0 and in_trade:
        in_trade = False
        exit_price = df['price'].iloc[i]
        pnl = amount_to_trade * (entry_price - exit_price) / exit_price
        cash += amount_to_trade + pnl
        trades.append(pnl)
    
    # Record balance
    balance_history.append(cash + (amount_to_trade + amount_to_trade * (entry_price - df['price'].iloc[i]) / df['price'].iloc[i]) if in_trade else cash)

# Calculate metrics
total_pnl = sum(trades)
total_roi = (cash - initial_capital) / initial_capital * 100
num_trades = len(trades)
num_profitable_trades = len([trade for trade in trades if trade > 0])
num_losing_trades = len([trade for trade in trades if trade < 0])
winrate = num_profitable_trades / num_trades * 100
max_drawdown = min(balance_history) - max(balance_history)

metrics = {
    'Total ROI': total_roi,
    'Total PnL': total_pnl,
    'Max Drawdown': max_drawdown,
    'Number of Trades': num_trades,
    'Number of Profitable Trades': num_profitable_trades,
    'Number of Losing Trades': num_losing_trades,
    'Winrate': winrate
}

short_entries = df[(df['Histogram'].shift(1) >= 0) & (df['Histogram'] < 0)]
short_exits = df[(df['Histogram'].shift(1) < 0) & (df['Histogram'] >= 0)]

# Create the main price chart for signals on price
fig1 = go.Figure()

# Add price data
fig1.add_trace(go.Scatter(x=df['Datetime'], y=df['price'], mode='lines', name='Price'))

# Add short entry points
fig1.add_trace(go.Scatter(x=short_entries['Datetime'], y=short_entries['price'], mode='markers', 
                          marker=dict(size=8, color='red', symbol='circle'), name='Short Entry'))

# Add short exit points
fig1.add_trace(go.Scatter(x=short_exits['Datetime'], y=short_exits['price'], mode='markers', 
                          marker=dict(size=8, color='green', symbol='circle'), name='Short Exit'))

# Set title and labels
fig1.update_layout(title='MACD Short Trading Signals on Price', xaxis_title='Date', yaxis_title='Price')

# Show the plot
fig1.show()

# Create the balance over time plot
fig2 = go.Figure()

# Assuming balance_history is already computed from the MACD strategy
# ...

fig2.add_trace(go.Scatter(x=df['Datetime'], y=balance_history, mode='lines', name='Balance'))

# Set title and labels
fig2.update_layout(title='Balance Over Time', xaxis_title='Date', yaxis_title='Balance')

# Show the plot
fig2.show()

from IPython.core.display import HTML

HTML('''
<script>
  function code_toggle() {
    setTimeout(function() {
      var cells = document.querySelectorAll('div.jp-Cell-inputWrapper');
      for (var i = 0; i < cells.length; i++) {
        cells[i].style.display = "none";
      }
      
      var tabs = document.querySelectorAll('lm-DockPanel-tabBar');
      for (var i = 0; i < tabs.length; i++) {
        tabs[i].style.display = "none";
      }
      
      var navElements = document.querySelectorAll('[role="navigation"]');
      for (var i = 0; i < navElements.length; i++) {
        navElements[i].style.display = "none";
      }
      
      var element = document.getElementById('jp-top-panel');
      if (element) {
        element.style.display = "none";
      }
      
      var complementaryElements = document.querySelectorAll('[role="complementary"]');
      for (var i = 0; i < complementaryElements.length; i++) {
        complementaryElements[i].style.display = "none";
      }
      
      var regionElements = document.querySelectorAll('[role="region"]');
      regionElements.forEach(function(element) {
        element.style.overflow = "hidden";
      });
      
      var leftStack = document.getElementById('jp-left-stack');
      if (leftStack) {
        leftStack.style.display = "none";
      }
    }, 0);
  }
  code_toggle();
</script>
''')